# Introduction à SPARK

## 1 Présentation


SPARK est un framework qui peut fonctionner de manière autonome ou être intégré à un cluster hadoop.
Ce qui est intéressant avec SPARK, le développeur peut se concentrer sur son application métier sans se soucier des problématiques lié aux traitements distribués.

Vous pouvez voir  SPARK comme une machine virtuelle composée de nombreuses ressources informatiques (CPU et RAM) qui exécute votre code.

Pour rendre transparente la logique cachée derrière l'informatique distribuée, SPARK utilise le concept de Resilient Distributed Dataset (RDD).
Un RDD est une API qui vous permet d'interagir avec vos données de manière unifiée et fournit de nombreuses fonctions pour faciliter la programmation.

Nous pouvons diviser l'API RDD en deux parties:
* Transformation:
Comme son nom l'indique, une transformation est un moyen de transformer vos données en quelque chose d'autre. Par exemple, si vous devez filtrer des données, nous transformons la source de données en filtrant les données, ce qui donnera un nouveau RDD.
Notez qu'une source de données (RDD) est immuable, vous ne pouvez donc pas en modifier le contenu. Pour modifier un RDD, l'astuce est d'appliquer vos modifications via des transformations qui seront contenues dans un nouveau RDD que vous pourrez sauvegarder.
La liste des transformations possibles est disponible sur le site d'apache Spark :
https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

* Action :
SPARK est fénéant/lazyness, cela signifie que SPARK ne fait rien tant que le programme n’exécute pas une  action. Par exemple, la fonction count()  est une action qui comptabilie le nombre de n-uplets dans le RDD.
Vous trouverez l'ensemble des actions possibles sur le site de Spark :
https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

Ce laboratoire vous présentera comment développer une application SPARK avec Python.
La librairie python pour Spark est détaillée ici :
https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD

Bien entendu, vous pouvez consulter le site Web de SPARK pour obtenir plus de détails: https://spark.apache.org/docs/latest/index.html


Alors, commençons maintenant quelques exemples pour devenir familier avec SPARK

Dans ce TP, nous explorerons les transformations et actions courantes fournies par Spark:

## 2 Pré-requis

### Démarrer Spark


Pour démarrer votre cluster Spark, si cela n'a pas déjà été fait, vous devrez ouvrir un terminal puis exécuter la commande suivante :

Vérifiez que les containers smaster, sworker1 et sworker2 sont démarrés :

### Connexion à Spark

Nous allons à présent établir une connexion avec notre cluster Spark.

Le notebook sera le driver lequel soumettra notre programme sur notre cluster Spark.

Nous utiliserons ce qu'on appelle un sparkContext pour identifier l'emplacement du cluster Spark.

Pour utiliser l'API RDD Spark, on crée un objet de connexion au cluster Spark :


In [1]:
# Attention le driver ne peut instancier qu'un seul SparkContext
# Si vous exécuter plusieurs fois cette cellule vous obtiendrez une erreur:
# Cannot run multiple SparkContexts at once;
# Pensez bien à fermer le Sparkcontext à la fin de chaque TP avec la commande sc.stop()

from pyspark import SparkContext

sc = SparkContext("spark://smaster:7077", "TPSPARK")


Pour vérifier le point d'accés au cluster : 


In [2]:
sc

<SparkContext master=spark://smaster:7077 appName=TPSPARK>

## 2 - Débuter avec Spark

Pour faciliter la prise en main de Spark, nous allons travailler avec de petits jeux de données.
Nous utiliserons des RDD contenant de 1 à 5 valeurs numériques mais il faut considérer que chaque valeur peut être une ligne d'un fichier.

### Collect / take / first / last

Renvoye tous / certains / le premier  éléments du RDD au driver dans une simple liste.

In [3]:

nums= sc.parallelize([1,2,3,4,5])
print(type(nums))

print( 'Collecte tous les éléments', nums.collect())
print( 'Collecte une partie des éléments', nums.take(3))
print( 'Collecte le 1er élément', nums.first())

<class 'pyspark.rdd.RDD'>
Collecte tous les éléments [1, 2, 3, 4, 5]
Collecte une partie des éléments [1, 2, 3]
Collecte le 1er élément 1


# Les Transformations

## MAP

La méthode map prend une fonction en entrée et l'applique à chaque élément du RDD source pour créer un nouveau RDD.
La fonction d'entrée doit prendre un seul paramètre d'entrée et renvoyer une valeur.

In [5]:
nums= sc.parallelize([1,2,3])

print("Type de nums : ", type(nums))
pluOneRDD = nums.map(lambda x : x+1)
print("plusOneRDD is new RDD")
print("Type de pluOneRDD : ", type(pluOneRDD))
pluOneRDD.collect()

Type de nums :  <class 'pyspark.rdd.RDD'>
plusOneRDD is new RDD
Type de pluOneRDD :  <class 'pyspark.rdd.PipelinedRDD'>


[2, 3, 4]

Une autre manière d'écrire ce programme :

In [6]:
nums= sc.parallelize([1,2,3])

def plus1(x):
    if( x > 2 ):
        s = x*x
    else:
        s = 1
    return s

# on  appelle la fonction plus1 directement dans la phase map
pluOneRDD = nums.map(lambda x : plus1(x))

print("map with lambda function")
print(pluOneRDD.collect())

map with lambda function
[1, 1, 9]


## Exercice 1

Tranformer la collection suivante [“John”, “Fred”, “Anna”, “James”] afin de retourner le nombre de caractères de chaque élément :

In [7]:
x = sc.parallelize(["John", "Fred", "Anna", "James"])
# Complétez le code
y = x.map( lambda x : len(x))
y.collect()

[4, 4, 4, 5]

## FILTER

Le filtre prend une fonction booléenne en entrée et l'applique à chaque élément du RDD source pour créer un nouveau RDD.
Une fonction booléenne prend une entrée et renvoie true ou false.
La méthode de filtrage renvoie un nouveau RDD formé en sélectionnant uniquement les éléments pour lesquels la fonction booléenne en entrée a renvoyé la valeur true.
Ainsi, le nouveau RDD contient un sous-ensemble des éléments du RDD d'origine.

In [8]:
nums= sc.parallelize([1,2,3])
myFilteredRDD = nums.filter(lambda x : x > 1)
myFilteredRDD.collect()

[2, 3]

## Exercise 2:

Changez la collection pour ne conserver que les nombres pairs et calculez le carré:

In [9]:
nums= sc.parallelize([1,2,3])
# Complétez le code
even= nums.filter(lambda x : x % 2 == 0).map(lambda x: x*x)
even.collect()

[4]

## FLATMAP

FlatMap prend une fonction d'entrée, qui renvoie une séquence pour chaque élément d'entrée qui lui est transmis.
La méthode flatMap renvoie un nouveau RDD formé en aplatissant cette collection de séquences.

In [10]:
nums= sc.parallelize([1,2,3])
mapRDD = nums.map(lambda x : (x,x*x,x*x*x))
flatMapRDD = mapRDD.flatMap(lambda x : x )
print("Display MapRDD :")
print(mapRDD.collect())

print("\nDisplay flatMapRDD :")
print(flatMapRDD.collect())

Display MapRDD :
[(1, 1, 1), (2, 4, 8), (3, 9, 27)]

Display flatMapRDD :
[1, 1, 1, 2, 4, 8, 3, 9, 27]


## Exercice 

Comptez le nombre de mots contenu dans l'ensemble des lignes faisant référence au "spam":


In [11]:
# Indice: vous utiliserez la fonction flatMap
data = sc.parallelize(["Spark c'est bien","il peut nous aider à trouver les spams","spam viagra.com"])
mapRDD = data.filter(lambda x: "spam" in x).flatMap(lambda x : x.split())
mapRDD.count()

10

## UNION

Union prend un RDD en entrée et retourne un nouveau RDD contenant l'union des éléments dans le RDD source et le RDD qui lui est transmis en tant qu'entrée.

In [12]:
x = sc.parallelize([[1,2],3])
y = sc.parallelize([1,2,3])
z = x.union(y)
z.collect()

[[1, 2], 3, 1, 2, 3]

## INTERSECTION 

La méthode intersection prend un RDD en entrée et retourne un nouveau RDD contenant l'intersection des éléments du RDD source et du RDD qui lui est transmis en tant qu'entrée.

In [13]:
x = sc.parallelize([1,2,3])
y = sc.parallelize([4,6,3])
z = x.intersection(y)
z.collect()

[3]

## La transformation subtract 

La méthode subtract prend un RDD en entrée et retourne un nouveau RDD contenant des éléments dans le RDD source mais pas dans le RDD en entrée.

In [14]:
x = sc.parallelize([1,2,3])
y = sc.parallelize([4,6,3])
z = x.subtract(y)
z.collect()

[1, 2]

## La transformation Distinct 

La méthode distincte d'un RDD renvoie un nouveau RDD contenant les éléments distincts dans le RDD source.

In [15]:
x = sc.parallelize([1,2,3,4,5,3,2,4])
y = x.distinct()
y.collect()

[2, 4, 1, 3, 5]

## La transformation Join 


La méthode de "join" prend un RDD de paires clé-valeur en entrée et effectue une jointure interne sur les RDD source et en entrée.

In [16]:
x = sc.parallelize([("TWTR", "Twitter"), ("GOOG", "Google"), ("AAPL", "Apple")])
y = sc.parallelize([("TWTR", 36), ("GOOG", 532), ("AAPL", 127)])
z = x.join(y).map(lambda x : x[1])
z.collect()

[('Google', 532), ('Twitter', 36), ('Apple', 127)]

## La transformation KeyBy

Créez un RDD Pair,  formant une paire pour chaque article du RDD original.
La clé de la paire est calculée à partir de la valeur via une fonction fournie par l'utilisateur.

In [17]:
x = sc.parallelize(range(0,5)).keyBy(lambda x: x*x)
x.collect()

[(0, 0), (1, 1), (4, 2), (9, 3), (16, 4)]

## Exercice 4


Créez un RDD à partir de cette liste, puis utilisez .keyBy pour créer une paire RDD avec: 
['New York, NY', 'Philadelphia, PA', 'Denver, CO', 'San Francisco, CA'])

Résultat :
[('New York', ' NY'), ('Philadelphia', ' PA'), ('Denver', ' CO'), ('San Francisco', ' CA')]

In [18]:
y = sc.parallelize(['New York, NY', 'Philadelphia, PA', 'Denver, CO', 'San Francisco, CA'])

# Complétez le code
y.keyBy(lambda x : x.split(",")[0]).map(lambda a : (a[0],a[1].split(",")[1])).collect()



[('New York', ' NY'),
 ('Philadelphia', ' PA'),
 ('Denver', ' CO'),
 ('San Francisco', ' CA')]

## Transformation leftOuterJoin / rightOuterJoin

Effectue une jointure externe gauche / droite à l'aide de deux RDD clé-valeur. Veuillez noter que les touches doivent être généralement comparables pour que cela fonctionne correctement.

In [19]:
x = sc.parallelize([("a", 1), ("b", 2), ("c", 2)])
y = sc.parallelize([("a", 3), ("a", 4), ("b", 5)] )
z = x.leftOuterJoin(y)
print(z.collect())

[('b', (2, 5)), ('c', (2, None)), ('a', (1, 4)), ('a', (1, 3))]


## Exercice 5


Créer une paire RDD renvoyant tous les mots par longueur pour les deux listes:

list1 = ["dog","cat","gnu","salmon","rabbit","turkey","wolf","bear","bee"]
list2 = [“dog”, “salmon”, “salmon”, “rat”, “elephant”]

Notez que le programme devra renvoyer au minimum tous les éléments de la list1.

In [20]:
list2 = sc.parallelize(['dog', 'salmon', 'salmon', 'rat', 'elephant'])
list1 = sc.parallelize(['dog','cat','gnu','salmon','rabbit','turkey','wolf','bear','bee'])

# Complétez le code
z = list1.keyBy(lambda x : len(x))
v = list2.keyBy(lambda x : len(x))
w = z.leftOuterJoin(v)
w.collect()

[(4, ('wolf', None)),
 (4, ('bear', None)),
 (6, ('salmon', 'salmon')),
 (6, ('salmon', 'salmon')),
 (6, ('rabbit', 'salmon')),
 (6, ('rabbit', 'salmon')),
 (6, ('turkey', 'salmon')),
 (6, ('turkey', 'salmon')),
 (3, ('dog', 'dog')),
 (3, ('dog', 'rat')),
 (3, ('cat', 'dog')),
 (3, ('cat', 'rat')),
 (3, ('gnu', 'dog')),
 (3, ('gnu', 'rat')),
 (3, ('bee', 'dog')),
 (3, ('bee', 'rat'))]

## La transformation PartitionBy

Renvoie un nouveau RDD avec le nombre spécifié de partitions, en plaçant les éléments d'origine dans la partition renvoyée par un partitionneur fourni par l'utilisateur.

In [21]:
rdd = sc.parallelize([('J', "James"), ('F', "Fred"), ('A', "Anna"), ('J', "John")])

rdd1 = rdd.partitionBy(2)
rdd2 = rdd.partitionBy(3)

print("2 partitions : ",rdd1.glom().collect())
print("3 partitions : ",rdd2.glom().collect())


2 partitions :  [[('J', 'John'), ('J', 'James')], [('F', 'Fred'), ('A', 'Anna')]]
3 partitions :  [[('J', 'James'), ('J', 'John')], [('A', 'Anna')], [('F', 'Fred')]]


## La transformation Zip

Renvoye un nouveau fichier RDD contenant des paires dont la clé est l’élément du fichier original, et dont la valeur correspond à l’élément correspondant de cet élément (même partition, même index) dans un deuxième fichier RDD.

![title](http://i.imgur.com/5J0lg6g.png)



In [22]:
x = sc.parallelize(range(0,5))
y = sc.parallelize(range(1000, 1005))
x.zip(y).collect()
[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]

[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]

# Les actions

Les actions calculent un résultat (par exemple, des données numériques ou créer une structure de données non RDD) ou produire un effet secondaire, tel que l'écriture d'une sortie sur un disque.
___________________

## L'action Reduce 

Comptez le nombre de mots pour cette liste avec les fonctions map et reduce:

[‘Spark s est vraiement facile’,‘Qu en pensez vous ?’]

In [23]:
from operator import add
x = sc.parallelize(['Spark s est vraiement facile','Qu en pensez vous ?'])
y = x.map(lambda x : x.split(' ')).flatMap(lambda x : x).map(lambda x : 1)
y.reduce(add)

10

## Les actions : Count, Max, Min, Sum, Mean, Variance, Stdev, stats 

In [24]:
x = sc.parallelize(range(10))

print("count",x.count())
print("max",x.max())
print("min",x.min())
print("sum",x.sum())
print("mean",x.mean())
print("variance",x.variance())
print("stdev",x.stdev())
print("stat", x.stats())

count 10
max 9
min 0
sum 45
mean 4.5
variance 8.25
stdev 2.8722813232690143
stat (count: 10, mean: 4.5, stdev: 2.8722813232690143, max: 9, min: 0)


## L'action CountByKey 

CountByKey : Retourne une liste des clés et compte leurs occurrences dans le RDD


In [25]:
x = sc.parallelize([('J', "James"), ('F', "Fred"), ('A', "Anna"), ('J', "John")])
y = x.countByKey()

print(y)


defaultdict(<class 'int'>, {'J': 2, 'F': 1, 'A': 1})


## ReduceByKey

Regroupe tous les éléments du RDD en appliquant une fonction utilisateur par paire aux éléments et aux résultats partiels et renvoyez un résultat au pilote.

In [26]:
from operator import add
z = sc.parallelize([(1,20),(1,30),(3,60),(3,20),(3,20)])

result = z.reduceByKey(add)
result.collect()


[(3, 100), (1, 50)]

## Exercice 6

Comptez le nombre de mots pour cette liste avec la fonction map et reduce :

In [27]:
from operator import add
x = sc.parallelize(['Spark s est vraiement facile','Qu en pensez vous ?'])
# Complétez le code
y = x.map(lambda x : x.split(' ')).flatMap(lambda x : x).map(lambda x : 1)
y.reduce(add)

10

Vous pouvez à présent fermer le SparkContext.

In [28]:
sc.stop()